# RNN test

## ref: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb


In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from functools import partial
from tensorflow.keras import models, layers

In [2]:
def build_rnn_model():
    # Each image is 91x91, and there are 146 such images per sequence
    time_steps = 146  # Number of images per sequence
    features_per_image = 91 * 91  # Each image is flattened into a vector of this size
    
    # Define the input shape for the RNN
    input_shape = (time_steps, features_per_image)
    
    # Create the model
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.LSTM(128, return_sequences=True))  # You can adjust the number of LSTM units
    model.add(layers.LSTM(128))  # Another LSTM layer, you can decide on return_sequences based on architecture needs
    model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# To create the model, just call the function
rnn_model = build_rnn_model()
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 146, 128)          4305920   
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 4,437,633
Trainable params: 4,437,633
Non-trainable params: 0
_________________________________________________________________


In [3]:
full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [4]:
# GAN Training Data Selection
gan_train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 4)
gan_test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in gan_train_ids_schiz]

gan_train_ids_control = random.sample(met_requirement_control_ids, 4)
gan_test_ids_control = [id for id in met_requirement_control_ids if id not in gan_train_ids_control]
gan_test_ids_control = random.sample(gan_test_ids_control,4)

''' data training for classifier '''
''' just use the same train set as GAN above '''

# Classifier Test Data Selection
classifier_test_ids = gan_test_ids_schiz + gan_test_ids_control

''' File loading '''
# Specify the directory and file pattern
directory_path = '4D/'
file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

# Construct the full path pattern
path_pattern = f'{directory_path}/{file_pattern}'

# Use glob to find all matching files
matching_files = glob.glob(path_pattern)

''' File loading for GAN Training and classifer '''
''' But this time we have 2 separate GANs, 1 train on schizoprenia and 1 train on control'''

#classifier_image_data = []
#classifier_labels = []  # 1 for schizophrenia, 0 for non-schizophrenia
gan_image_data_schiz = []
gan_image_data_control = []

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in gan_train_ids_schiz:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()
        

        if t1_data.shape[3] < 90:
            continue

        t1_data_collapsed = np.sum(t1_data, axis=1)
        print('shape of image after collapsed: ', t1_data_collapsed.shape)
        gan_image_data_schiz.append(t1_data_collapsed)

    if file_id in gan_train_ids_control:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue

        t1_data_collapsed = np.sum(t1_data, axis=1)
        gan_image_data_control.append(t1_data_collapsed)


print(f"Total GAN control loaded: {len(gan_image_data_control)}")
print(f"Total GAN schiz loaded: {len(gan_image_data_schiz)}")



'''Determine the maximum 3rd-dimension size '''
max_z_size_schiz = max(img.shape[2] for img in gan_image_data_schiz)
max_z_size_control = max(img.shape[2] for img in gan_image_data_control)

max_z_size = max(max_z_size_schiz,max_z_size_control)

''' normalization '''
image_data_normalized_schiz = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_schiz]
image_data_normalized_control = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_control]


''' padding of images data '''
# Pad each image to have a consistent z-dimension size
padded_data_schiz = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size - img.shape[2])), mode='constant') for img in image_data_normalized_schiz]
padded_data_control = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size - img.shape[2])), mode='constant') for img in image_data_normalized_control]


# Now convert the padded data list to a numpy array
padded_data_array_schiz = np.array(padded_data_schiz)
padded_data_array_control = np.array(padded_data_control)


''' loading the data for WGAN training '''
train_images_schiz = padded_data_array_schiz
train_images_control = padded_data_array_control

# Define batch size
batch_size = 10


train_dataset_schiz = tf.data.Dataset.from_tensor_slices((train_images_schiz)).shuffle(len(train_images_schiz)).batch(batch_size)
train_dataset_control = tf.data.Dataset.from_tensor_slices((train_images_control)).shuffle(len(train_images_control)).batch(batch_size)

shape of image after collapsed:  (91, 91, 124)
shape of image after collapsed:  (91, 91, 111)
shape of image after collapsed:  (91, 91, 143)


In [ ]:
# Flatten each 2D image slice and prepare it for RNN input
train_images_schiz = np.reshape(padded_data_array_schiz, (len(padded_data_array_schiz), 146, 91*91))
train_images_control = np.reshape(padded_data_array_control, (len(padded_data_array_control), 146, 91*91))

# Create labels
labels_schiz = np.ones(len(train_images_schiz))
labels_control = np.zeros(len(train_images_control))

# Combine the datasets
train_images = np.concatenate((train_images_schiz, train_images_control), axis=0)
train_labels = np.concatenate((labels_schiz, labels_control), axis=0)

# Shuffle the dataset
indices = np.arange(train_images.shape[0])
np.random.shuffle(indices)
train_images = train_images[indices]
train_labels = train_labels[indices]

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)


In [ ]:
# epochs = 20

# # Training loop
# for epoch in range(epochs):
#     for images, labels in train_dataset:
#         with tf.GradientTape() as tape:
#             predictions = rnn_model(images, training=True)
#             # Squeeze the predictions to remove the extra dimension
#             predictions = tf.squeeze(predictions, axis=1)
#             loss = tf.keras.losses.binary_crossentropy(labels, predictions)
#         gradients = tape.gradient(loss, rnn_model.trainable_variables)
#         rnn_model.optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))
#     print(f"Epoch {epoch+1}, Loss: {loss.numpy().mean()}")

Epoch 1, Loss: 0.6909060478210449
Epoch 2, Loss: 0.6627525091171265
Epoch 3, Loss: 0.6003762483596802
Epoch 4, Loss: 0.5706890821456909
Epoch 5, Loss: 0.5058412551879883
Epoch 6, Loss: 0.5379464030265808
Epoch 7, Loss: 0.5105340480804443
Epoch 8, Loss: 0.3899977207183838
Epoch 9, Loss: 0.33264148235321045
Epoch 10, Loss: 0.26590174436569214
Epoch 11, Loss: 0.25205886363983154
Epoch 12, Loss: 0.1629202961921692
Epoch 13, Loss: 0.09086810052394867
Epoch 14, Loss: 0.1215558871626854
Epoch 15, Loss: 0.8470641374588013
Epoch 16, Loss: 0.09229753166437149
Epoch 17, Loss: 0.06965304911136627
Epoch 18, Loss: 0.07012609392404556
Epoch 19, Loss: 0.061629049479961395
Epoch 20, Loss: 0.05886310338973999


In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

# Define number of epochs
epochs = 20

# Training loop
for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for images, labels in train_dataset:
        with tf.GradientTape() as tape:
            predictions = rnn_model(images, training=True)
            loss = tf.keras.losses.binary_crossentropy(labels, tf.squeeze(predictions, axis=1))
        
        gradients = tape.gradient(loss, rnn_model.trainable_variables)
        rnn_model.optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))
        
        # Update the metrics
        train_loss.update_state(loss)
        train_accuracy.update_state(labels, tf.squeeze(predictions, axis=1))
    
    # Print the loss and accuracy for the current epoch
    print(f"Epoch {epoch + 1}, Loss: {train_loss.result().numpy()}, Training Accuracy: {train_accuracy.result().numpy()}")


Epoch 1, Loss: 0.051245108246803284, Accuracy: 1.0
Epoch 2, Loss: 0.04236660152673721, Accuracy: 1.0
Epoch 3, Loss: 0.0335816815495491, Accuracy: 1.0
Epoch 4, Loss: 0.02804667130112648, Accuracy: 1.0
Epoch 5, Loss: 0.02422485500574112, Accuracy: 1.0
Epoch 6, Loss: 0.01890498213469982, Accuracy: 1.0
Epoch 7, Loss: 0.08708363771438599, Accuracy: 1.0
Epoch 8, Loss: 1.2482094764709473, Accuracy: 0.75
Epoch 9, Loss: 1.441906213760376, Accuracy: 0.75
Epoch 10, Loss: 1.4340698719024658, Accuracy: 0.75


In [ ]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0] / image.shape[0],
        new_shape[1] / image.shape[1],
        new_shape[2] / image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation


test_image_data = []
test_labels = []

test_ids = classifier_test_ids  # List of IDs to filter test data

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in test_ids:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue
        
        t1_data_collapsed = np.sum(t1_data, axis=1)  # Summing over one axis

        # Resize the collapsed data
        processed_image = resize_image(t1_data_collapsed, (91, 91, 146))

        # Normalize the processed image
        processed_image_normalized = (processed_image - np.min(processed_image)) / (np.max(processed_image) - np.min(processed_image)) * 2 - 1

        # Reshape for RNN input: Flatten each 91x91 image to a single vector and treat each as a timestep
        processed_image_flattened = np.reshape(processed_image_normalized, (146, 91*91))

        test_image_data.append(processed_image_flattened)
        label = 1 if file_id in met_requirement_schizophrenia_ids else 0
        test_labels.append(label)

# Convert to numpy arrays for easier handling in TensorFlow
test_images_array = np.array(test_image_data)
test_labels_array = np.array(test_labels)

# Create a TensorFlow dataset from the numpy arrays
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)


In [ ]:
# Evaluate the model
loss, accuracy = rnn_model.evaluate(test_dataset)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

6/6 [==============================] - 1s 50ms/step - loss: 1.6054 - accuracy: 0.5085
Test loss: 1.6054370403289795, Test accuracy: 0.508474588394165


In [ ]:
# Predict the probabilities
predictions = rnn_model.predict(test_dataset)
# Convert probabilities to class labels
predicted_labels = (predictions > 0.5).astype(int)

# Assuming test_labels_array contains your actual labels
actual_labels = test_labels_array

# Now you might want to compare these predicted_labels with the actual labels (test_labels)
# to compute the confusion matrix, classification report, etc.
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(actual_labels, predicted_labels.flatten()))
print(classification_report(actual_labels, predicted_labels.flatten()))

6/6 [==============================] - 1s 49ms/step
[[ 1  3]
 [26 29]]
              precision    recall  f1-score   support

           0       0.04      0.25      0.06         4
           1       0.91      0.53      0.67        55

    accuracy                           0.51        59
   macro avg       0.47      0.39      0.37        59
weighted avg       0.85      0.51      0.63        59



In [ ]:
for actual, predicted in zip(actual_labels, predicted_labels):
    print(f'Actual: {actual}, Predicted: {predicted}')

Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 0, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 0, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [1]
Actual: 0, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, P